# Using Redpanda to test FastKafka

## What is Redpanda?

Redpanda is a drop-in replacement for Kafka. Most of the Kafka tools work out of the box with Redpanda.

From [redpanda.com](https://redpanda.com/):

> Redpanda is a Kafka®-compatible streaming data platform that is proven to be 10x faster and 6x lower in total costs. It is also JVM-free, ZooKeeper®-free, Jepsen-tested and source available.

Some of the advantages of Redpanda over Kafka are

1. A single binary with built-in everything, no ZooKeeper® or JVM needed.
2. Costs upto 6X less than Kafka.
3. Up to 10x lower average latencies and up to 6x faster Kafka transactions without compromising correctness.

To learn more about Redpanda, please visit their [website](https://redpanda.com/) or checkout this [blog post](https://redpanda.com/blog/redpanda-vs-kafka-performance-benchmark) comparing Redpanda and Kafka's performance benchmarks.

## Example repo

A sample fastkafka-based library that uses Redpanda for testing, based on this guide, can be found [here](https://github.com/airtai/sample_fastkafka_with_redpanda).

## Testing FastKafka with Redpanda

To test a FastKafka project with Redpanda, we need the following:

1. A Python file, project, or library built using FastKafka.
2. A test file that uses FastKafka's `Tester` with Redpanda.

### Creating FastKafka code

Let's create a `FastKafka` based application and write to the `application.py` file based on the [tutorial](/#tutorial)

```python
# content of the "application.py" file

from pydantic import BaseModel, NonNegativeFloat, Field

class IrisInputData(BaseModel):
    sepal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal length in cm"
    )
    sepal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Sepal width in cm"
    )
    petal_length: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal length in cm"
    )
    petal_width: NonNegativeFloat = Field(
        ..., example=0.5, description="Petal width in cm"
    )


class IrisPrediction(BaseModel):
    species: str = Field(..., example="setosa", description="Predicted species")

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Iris predictions",
    kafka_brokers=kafka_brokers,
    bootstrap_servers="localhost:9092",
)

iris_species = ["setosa", "versicolor", "virginica"]

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: IrisInputData):
    global model
    species_class = model.predict([
          [msg.sepal_length, msg.sepal_width, msg.petal_length, msg.petal_width]
        ])[0]

    to_predictions(species_class)


@kafka_app.produces(topic="predictions")
def to_predictions(species_class: int) -> IrisPrediction:
    prediction = IrisPrediction(species=iris_species[species_class])
    return prediction

```

### Writing tests with Redpanda

Let's create a `Tester` based test code as explained in [tutorial](/#tutorial). Instead of Kafka(which is the default broker), we will use [Redpanda broker](/api/fastkafka/testing/LocalRedpandaBroker/).

```python
# content of the "test.py" file

import pytest
from fastkafka.testing import Tester

from application import IrisInputData, kafka_app, IrisPrediction

msg = IrisInputData(
    sepal_length=0.1,
    sepal_width=0.2,
    petal_length=0.3,
    petal_width=0.4,
)

@pytest.mark.asyncio
async def test():
    # Start Tester app and create a local Redpanda broker for testing
    async with Tester(kafka_app).using_local_redpanda(tag="v23.1.2", listener_port=9092) as tester:
        # Send IrisInputData message to input_data topic
        await tester.to_input_data(msg)

        # Assert that the kafka_app responded with IrisPrediction in predictions topic
        await tester.awaited_mocks.on_predictions.assert_awaited_with(
            IrisPrediction(species="setosa"), timeout=2
        )
```

Save the above code to a file called `test.py` and run the tests by executing the following command,

```cmd
pytest test.py
```

This will start a Redpanda broker with Docker and will start executing tests. The output of the command is

```cmd
(venv) fastkafka@airt-ai:~/dev/sample_fastkafka_with_redpanda$ pytest
================================================================================ test session starts =================================================================================
platform linux -- Python 3.10.6, pytest-7.2.2, pluggy-1.0.0
rootdir: /home/kumaran/dev/sample_fastkafka_with_redpanda, configfile: pytest.ini, testpaths: test.py
plugins: asyncio-0.21.0, anyio-3.6.2
asyncio: mode=strict
collected 1 item                                                                                                                                                                     

test.py .                                                                                                                                                                      [100%]

================================================================================= 1 passed in 7.17s ==================================================================================
(venv) fastkafka@airt-ai:~/dev/sample_fastkafka_with_redpanda$
```

Running the tests with the Redpanda broker ensures that your code is working correctly with a real Kafka-like message broker, making your tests more reliable. 